In [10]:
import torch
import json
from utils import ToyTransformer, ModelConfig

# we want to load in the config & weights 
model_name = "attention_only_1L"

# load in the config
with open(f'{model_name}_config.json', 'r') as f:
    config = json.load(f)
# turn config into a dot dict
config = ModelConfig(**config)
# load in the weights
model = ToyTransformer(config)
state_dict = torch.load(f'toy_transformer_{model_name}.pt')
model.load_state_dict(state_dict)

# we want to load in the data

/tmp/ipykernel_3021/4289467497.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f'toy_transformer_{model_name}.pt')


<All keys matched successfully>

In [11]:
model

ToyTransformer(
  (embed): Embedding(50257, 512)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): QuadraticAttention(
      (rotary): Rotary()
      (norm): RMSNorm((64,), eps=None, elementwise_affine=True)
      (mask): Mask()
      (q): Linear(in_features=512, out_features=512, bias=True)
      (k): Linear(in_features=512, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=512, bias=True)
      (o): Linear(in_features=512, out_features=512, bias=False)
    )
  )
  (head): Linear(in_features=512, out_features=50257, bias=False)
)

In [22]:
E = model.embed.weight.data
Q = model.layers[0].q.weight.data
K = model.layers[0].k.weight.data
V = model.layers[0].v.weight.data
O = model.layers[0].o.weight.data
U = model.head.weight.data

print(f"Shape of E: {E.shape}")
print(f"Shape of Q: {Q.shape}")
print(f"Shape of K: {K.shape}")
print(f"Shape of V: {V.shape}")
print(f"Shape of O: {O.shape}")

attn_head_idx = 0
d_head = model.layers[0].d_head

q = Q[attn_head_idx * d_head:(attn_head_idx + 1) * d_head]
k = K[attn_head_idx * d_head:(attn_head_idx + 1) * d_head]
v = V[attn_head_idx * d_head:(attn_head_idx + 1) * d_head]
o = O[attn_head_idx * d_head:(attn_head_idx + 1) * d_head]

qk = q.T @ k
ov = v.T @ o

qk_circuit = E @ qk @ E.T
ov_circuit = U @ ov @ E.T

Shape of E: torch.Size([50257, 512])
Shape of Q: torch.Size([512, 512])
Shape of K: torch.Size([512, 512])
Shape of V: torch.Size([512, 512])
Shape of O: torch.Size([512, 512])


In [23]:
from tinymodel import TinyModel, tokenizer

lm = TinyModel()

# for inference
tok_ids, padding_mask = tokenizer(['Once upon a time', 'In the forest'])
logprobs = lm(tok_ids)

# Get SAE/transcoder acts
# See 'SAEs/Transcoders' section for more information.
feature_acts = lm['M1N123'](tok_ids)
all_feat_acts = lm['M2'](tok_ids)

# Generation
lm.generate('Once upon a time, Ada was happily walking through a magical forest with')

# To decode tok_ids you can use
tokenizer.decode(tok_ids)

ImportError: cannot import name 'TinyModel' from 'tinymodel' (/venv/main/lib/python3.10/site-packages/tinymodel/__init__.py)

In [ ]:
from tokenization.tokenization import tokenizer

10000